In [4]:
# %pip install nbformat

In [5]:
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

In [6]:
%run ml_project_normalized.ipynb

Note: you may need to restart the kernel to use updated packages.


/u/nneoma/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.17.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
Done running ml_project.ipynb.


In [7]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE, SMOTENC

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.svm import LinearSVC

def train_SVM_classifier(X_train, y_train, X_test):

    class_weights = get_class_weights(y_train)

    pipeline = Pipeline([
        ('smote', SMOTENC(categorical_features=[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11], random_state=42)),# 'intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed', 'is_mix', 'primary_color'], random_state=42)),
        ('freq', FunctionTransformer(apply_freq_encode, validate=False)),
        ('onehot', OneHotEncoder( handle_unknown='ignore', sparse_output=True)),
        ('linsvc', LinearSVC(class_weight=class_weights, max_iter=10000, dual=False))
    ])

    # Set up parameter distributions for XGBoost.
    param_distributions = {
    'linsvc__C': [0.01, 0.1, 1, 2, 4, 8, 10],
    'linsvc__loss': ['squared_hinge', 'hinge'],  # 'hinge' is okay too, but 'squared_hinge' usually works better
   # 'linsvc__penalty': ['l2'],  # 'l1' can only be used with dual=False and 'liblinear' solver
    'linsvc__max_iter': [1000, 5000, 10000],
    'linsvc__multi_class': ['ovr', 'crammer_singer'],  # try both strategies
    }
    # Perform hyperparameter search using RandomizedSearchCV.
    randomized_search = RandomizedSearchCV(
        estimator=pipeline, 
        param_distributions=param_distributions,
        n_iter=1,
        cv=skf, 
        scoring=balanced_acc_scorer, 
        verbose=3,
    )


        
    randomized_search.fit(X_train, y_train)
    
    print('Best parameters:', randomized_search.best_params_)
    print('Best cross-validation accuracy:', randomized_search.best_score_)
    
    cv_scores = cross_val_score(randomized_search.best_estimator_, X_train, y_train, cv=skf, verbose=3, scoring=balanced_acc_scorer)
    print('Generalization Balanced accuracy (via cross_val_score):', cv_scores.mean())

    # Make predictions on the test set using the best estimator.
    test_predictions = randomized_search.predict(X_test)
        
    return randomized_search.best_estimator_, test_predictions


In [9]:
df_train = engineer_time(df_train)
df_train = time_as_cyclical(df_train)
df_train.head()

,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,is_mix,primary_color,outcome_type,intake_year,is_weekend,hour_sin,hour_cos,month_sin,month_cos
0,Stray,Normal / Behavior,Dog,Spayed Female,8.000000,English Springer Spaniel,0,white,Return to Owner,2015,1,1.224647e-16,-1.000000e+00,-0.500000,-0.866025
1,Stray,Normal / Behavior,Dog,Intact Male,0.916667,Basenji,1,sable,Return to Owner,2016,0,-1.000000e+00,-1.836970e-16,0.866025,-0.500000
2,Public Assist,Normal / Behavior,Cat,Neutered Male,2.000000,Domestic Shorthair,0,orange,Transfer,2022,0,0.000000e+00,1.000000e+00,0.500000,-0.866025
3,Owner Surrender,Normal / Behavior,Dog,Neutered Male,2.000000,Labrador Retriever,1,chocolate,Return to Owner,2017,1,1.224647e-16,-1.000000e+00,0.866025,0.500000
4,Public Assist,Normal / Behavior,Dog,Neutered Male,6.000000,Great Dane,1,black,Return to Owner,2019,0,7.071068e-01,-7.071068e-01,0.866025,-0.500000


In [ ]:

# Cross Validation with Linear


 # categorical feature sarray will change as a i feautre engineer


X = df_train.drop(columns=['outcome_type'])
y = df_train['outcome_type']
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.20, random_state=42)


class_weights_cv = get_class_weights(y_train)

# Define which columns are categorical and which are numerical
#categorical_cols = ['intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed', 'is_mix', 'primary_color']
#numerical_cols = ['age_upon_intake', 'year', 'month', 'hour']  # Add any other numerical columns you may have

# Define preprocessing pipeline
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())  # Scale numerical features
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True))  # One hot encode categorical features
])

# Combine preprocessing steps for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, [4, 8, 10, 11, 12, 13]),
        ('cat', categorical_transformer, [0, 1, 2, 3, 5, 6, 7, 9, 10, 11])
    ]
)

# Create the full pipeline including SMOTE for oversampling, frequency encoding, and LinearSVC
pipeline = Pipeline([
    ('smote', SMOTENC(categorical_features=[0, 1, 2, 3, 5, 6, 7, 9, 10, 11], random_state=42)),  # Ensure correct categorical indices
    ('freq', FunctionTransformer(apply_freq_encode, validate=False)),  # Apply frequency encoding function
    ('preprocessor', preprocessor),  # Preprocessing step (scaling numerical, one-hot encoding categorical)
    ('linsvc', LinearSVC(class_weight=class_weights_cv, dual=False))
])

# Define the hyperparameter search space
param_distributions = {
    'linsvc__C': [4, 8, 10],
    'linsvc__loss': ['squared_hinge'],  # 'hinge' is okay too, but 'squared_hinge' usually works better
    'linsvc__max_iter': [1000, 5000, 10000],
    'linsvc__multi_class': ['ovr', 'crammer_singer'],  # try both strategies
}

# Perform hyperparameter search using RandomizedSearchCV
randomized_search = RandomizedSearchCV(
    estimator=pipeline, 
    param_distributions=param_distributions,
    n_iter=5,
    cv=skf,  # Ensure you've defined skf (StratifiedKFold) elsewhere
    scoring=balanced_acc_scorer,  # Ensure you've defined balanced_acc_scorer elsewhere
    verbose=3,
)

# Fit the model using the training data
randomized_search.fit(X_train, y_train)

# Print best parameters and best cross-validation score
print('Best parameters:', randomized_search.best_params_)
print('Best cross-validation accuracy:', randomized_search.best_score_)

# Perform cross-validation to estimate generalization accuracy
cv_scores = cross_val_score(randomized_search.best_estimator_, X_train, y_train, cv=skf, verbose=3, scoring=balanced_acc_scorer)
print('Generalization Balanced accuracy (via cross_val_score):', cv_scores.mean())

# Make predictions on the test set using the best estimator
test_predictions = randomized_search.predict(X_test)

classification_report_with_accuracy_score(y_test, test_predictions)


Fitting 5 folds for each of 5 candidates, totalling 25 fits


In [ ]:
'''

# Cross Validation with just regular SVC
from sklearn.svm import SVC

# Cross Validation
X = df_train.drop(columns=['outcome_type'])
y = df_train['outcome_type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

class_weights_cv = get_class_weights(y_train)

pipeline = Pipeline([
    ('smote', SMOTENC(categorical_features=[0, 1, 2, 3, 5, 6, 7, 9], random_state=42)),
    ('freq', FunctionTransformer(apply_freq_encode, validate=False)),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True)),
    ('svc', SVC(class_weight=class_weights_cv, probability=False))  # Set to True if you need proba
])

# Hyperparameters for SVC
param_distributions = {
    'svc__C': [0.01, 0.1, 1, 2, 4, 7, 8 , 9, 10],
    'svc__kernel': ['rbf', 'poly', 'sigmoid', 'precomputed'],  # Try a few kernels
    'svc__gamma': ['scale', 'auto'],
    'svc__degree': [2, 3],  # Only relevant for poly kernel
    'svc__max_iter': [1000, 5000, 10000, 25000]
}

# Randomized Search
randomized_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=10,
    cv=skf,
    scoring=balanced_acc_scorer,
    verbose=3
)

# Fit
randomized_search.fit(X_train, y_train)

# Output
print('Best parameters:', randomized_search.best_params_)
print('Best cross-validation accuracy:', randomized_search.best_score_)

cv_scores = cross_val_score(
    randomized_search.best_estimator_, X_train, y_train,
    cv=skf, verbose=3, scoring=balanced_acc_scorer
)
print('Generalization Balanced accuracy (via cross_val_score):', cv_scores.mean())

# Predict
test_predictions = randomized_search.predict(X_test)
classification_report_with_accuracy_score(y_test, test_predictions)
'''


Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [73]:
X_train.head()

,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,is_mix,primary_color,intake_year,is_weekend,hour_sin,hour_cos,month_sin,month_cos
107615,Stray,Normal / Behavior,Cat,Intact Female,0.083333,Domestic Shorthair,0,torbie,2021,0,0.500000,-8.660254e-01,-5.000000e-01,-0.866025
13002,Stray,Injured,Dog,Intact Male,2.000000,Akita/Australian Cattle Dog,0,brown brindle,2014,0,0.258819,-9.659258e-01,-8.660254e-01,-0.500000
32513,Public Assist,Normal / Behavior,Dog,Intact Female,1.000000,Rat Terrier,1,brown,2019,0,-0.707107,7.071068e-01,-5.000000e-01,-0.866025
79497,Stray,Normal / Behavior,Cat,Intact Female,0.083333,Domestic Shorthair,1,calico,2015,1,-1.000000,-1.836970e-16,1.224647e-16,-1.000000
88489,Stray,Normal / Behavior,Dog,Intact Female,0.333333,Labrador Retriever,0,black,2017,0,0.500000,-8.660254e-01,5.000000e-01,-0.866025
